### Imports

In [1]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from transformers import LlamaForCausalLM, LlamaTokenizer, TextStreamer
import huggingface_hub as hf_hub
from langchain.llms.base import LLM
from langchain import PromptTemplate, LLMChain
import torch
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import AgentType
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.embeddings import HuggingFaceInstructEmbeddings


### Loading embeddings and vicuna 13b

In [2]:
HF_embed_model = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

load INSTRUCTOR_Transformer
max_seq_length  512


In [3]:
huggingface_api_key = ''

hf_hub.login(token=huggingface_api_key)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [4]:

model_name = 'shrinath-suresh/vicuna-13b'

model = LlamaForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
    )

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Building langchain llm_chain

In [5]:
tokenizer = LlamaTokenizer.from_pretrained(model_name)

In [6]:
prompt_template = "Below is an instruction that describes a task. If question is related to pytorch Write a response that appropriately completes the request.\n\n### Instruction:\n{question}\n\n### Response:\n"

prompt = PromptTemplate(
            template=prompt_template, input_variables=["question"]
        )

## tool 1
wikipedia = WikipediaAPIWrapper()

In [7]:
class CustomLLM(LLM):
    def _call(self, prompt, stop=None) -> str:
        inputs = tokenizer([prompt], return_tensors="pt")

        response = model.generate(**inputs, max_new_tokens=2048)
        response = tokenizer.decode(response[0])
        return response

    @property
    def _identifying_params(self):
        return {"name_of_model": model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

llm = CustomLLM()

In [8]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

### Initializing tools

In [9]:
tools = [
    Tool(
        name="pytorch search",
        func=llm_chain.run,
        description="Use this to answer questions only related to pytorch",
        return_direct=True
    ),
    Tool(
        name="wikipedia search",
        func=wikipedia.run,
        description="Use this to search wikipedia for general questions",
        return_direct=True
    )
]


### Using embeddings to identify the right tool

In [11]:
docs = [Document(page_content=t.description, metadata={"index": i}) for i, t in enumerate(tools)]

vector_store = FAISS.from_documents(docs, HF_embed_model)

def get_tools(query):
    docs = vector_store.similarity_search_with_score(query)
    print(docs)
    return [tools[d[0].metadata["index"]] for d in docs]

### Tool Selection and Similarity Evaluation

In [13]:
vector_store.similarity_search_with_score("What is the weather in LA?")

[(Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}),
  1.0683217),
 (Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}),
  1.1081737)]

In [14]:
vector_store.similarity_search_with_score("Convert numpy to tensors?")

[(Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}),
  0.7281347),
 (Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}),
  1.0837196)]

In [15]:
vector_store.similarity_search_with_score("What is the capital of United State of America?")

[(Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}),
  1.0892316),
 (Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}),
  1.15314)]

In [23]:
vector_store.similarity_search_with_score("How to debug OOM issues?")

[(Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}),
  0.863247),
 (Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}),
  0.95337117)]

In [24]:
vector_store.similarity_search_with_score("What are the benefits of reading books?")

[(Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}),
  1.0881928),
 (Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}),
  1.1561345)]

In [25]:
vector_store.similarity_search_with_score("What is relu?")

[(Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}),
  0.9498791),
 (Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}),
  0.9604829)]

### Use Agents to fetch the answer

In [39]:

def get_result(query):
    tools = get_tools(query)
    agent_chain = initialize_agent([tools[0]], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True,max_iterations=1,return_intermediate_steps=True)
    answer = agent_chain(query)
#     print(answer['intermediate_steps'][0][0].log.split('Final Answer: ')[-1])
    return answer['intermediate_steps'][0][0].log.split('Final Answer: ')[-1]

### Sample Questions

In [45]:
query = 'which is the largest city in United States?'
get_result(query)

[(Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}), 1.1483727), (Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}), 1.2082007)]


> Entering new  chain...
Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

wikipedia search: Use this to search wikipedia for general questions

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [wikipedia search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: which is the largest city in United States?
Thought: I sho

'The largest city in the United States is New York City.</s><s>'

In [20]:
query = 'How to convert numpy to tensors?'
get_result(query)

[(Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}), 0.7369725), (Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}), 1.0988166)]


> Entering new  chain...
Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: How to convert numpy to tensors?
Thought: I should use the t

In [21]:
query = 'What is the capital of United State of America??'
get_result(query)

[(Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}), 1.0918921), (Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}), 1.1572859)]


> Entering new  chain...
Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

wikipedia search: Use this to search wikipedia for general questions

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [wikipedia search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What is the capital of United State of America??
Thought: 

In [22]:
query = 'How to debug OOM issues?'
get_result(query)

[(Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}), 0.8878783), (Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}), 0.97247136)]


> Entering new  chain...
Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: How to debug OOM issues?
Thought: I should check the memory

In [26]:
query = 'What are the benefits of reading books?'
get_result(query)

[(Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}), 1.0881928), (Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}), 1.1561345)]


> Entering new  chain...
Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

wikipedia search: Use this to search wikipedia for general questions

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [wikipedia search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What are the benefits of reading books?
Thought: I should 

In [27]:
query = 'What is relu?'
get_result(query)

[(Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}), 0.9498791), (Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}), 0.9604829)]


> Entering new  chain...
Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What is relu?
Thought: I should look up the definition of re

In [29]:
query = 'How to save the model?'
get_result(query)

[(Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}), 0.849853), (Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}), 0.969757)]


> Entering new  chain...
Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: How to save the model?
Thought: I should check the documentati

### Benchmarking

In [30]:
question_list = [
    "How do I check if PyTorch is using the GPU?",
    "Code to save the model?",
    "What does .view() do?",
    "Why do we need to call zero_grad()?",
    "How do I print the model summary?",
    "What is the purpose of PyTorch's grad_fn attribute in tensors?",
    "What is fsdp and how to use it in code?",
    "How train the model in multi-node",
    "How to write a custom dataset",
    "How should I use torch.compile properly??",
    "Why do we pack the sequences?",
    "Check the total number of parameters",
    "RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same",
    "Connection between loss.backward() and optimizer.step()",
    "How to improve model accuracy",
    "How to improve the training speed",
    "How to manage huge input tensor?",
    "How to fix RuntimeError 'Expected object of scalar type Float but got scalar type Double for argument'?",
    "how to do mixed precision training?",
    "How to avoid CUDA out of memory?"
]

In [42]:
from tqdm import tqdm
answer_list = []
for question in tqdm(question_list):
    result = get_result(question)
    answer_list.append(result)
    

  0%|                                                                                                                                                                                | 0/20 [00:00<?, ?it/s]

[(Document(page_content='Use this to answer questions only related to pytorch', metadata={'index': 0}), 0.5040008), (Document(page_content='Use this to search wikipedia for general questions', metadata={'index': 1}), 0.990008)]


> Entering new  chain...


  5%|████████▍                                                                                                                                                               | 1/20 [00:15<05:01, 15.84s/it]

Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: How do I check if PyTorch is using the GPU?
Thought: I should check the device on which the tensor is stored.
Action: [pytorch search]
Action Input: None
Observation: The output is a boolean indicating whether PyTorch is using the GPU or not.
...
Thought: I should check the device on which the tensor is stored.
Act

 10%|████████████████▊                                                                                                                                                       | 2/20 [00:33<05:04, 16.90s/it]

...

Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: How to fix RuntimeError 'Expected object of scalar type Float but got scalar type Double for argument'?
Thought: This error is usually caused by a mismatch in the data types of the inputs to a function.
Action: Check the data types of the inputs to the function and make sure they match.
Action Input: The inputs to 

 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋        | 19/20 [08:13<00:27, 27.98s/it]

Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: how to do mixed precision training?
Thought: I need to know more about mixed precision training
Action: [pytorch search] mixed precision training
Action Input: the model architecture and data type
Observation: the code for mixed precision training using the specified model architecture and data type
...
Thought: I 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [08:19<00:00, 24.99s/it]

Parsing LLM output produced both a final answer and a parse-able action: <s> Answer the following questions as best you can. You have access to the following tools:

pytorch search: Use this to answer questions only related to pytorch

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [pytorch search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: How to avoid CUDA out of memory?
Thought: Check the memory usage and distribution of data.
Action: Use torch.autograd.detect_anomaly()
Action Input: None
Observation: The function returns a dictionary with information about the memory usage and distribution of data.
...
Thought: I now know the final answer
Final An

In [44]:
for question, answer in zip(question_list, answer_list):
    print(f"Question: {question} \n Answer: {answer} \n")

Question: How do I check if PyTorch is using the GPU? 
 Answer: To check if PyTorch is using the GPU, you can use the `torch.cuda.is_available()` function to check if CUDA is available and then use `torch.cuda.current()` to check which GPU PyTorch is using.</s><s> 

Question: Code to save the model? 
 Answer: The code to save the model is "torch.jit.save(model, path)", the code to load the model is "torch.jit.load(path)", and the code to use the loaded model in evaluation mode is "model.eval()".</s><s> 

Question: What does .view() do? 
 Answer: The .view() function is used to change the shape, size and stride of a tensor without changing its data.</s><s> 

Question: Why do we need to call zero_grad()? 
 Answer: We need to call zero\_grad() to reset the gradients to their initial values before starting the optimization process.</s><s> 

Question: How do I print the model summary? 
 Answer: The model summary is printed</s><s> 

Question: What is the purpose of PyTorch's grad_fn attribut